## Investigating problematic voltage setpoints with pypowsybl-jupyter
- This notebook demonstrates how to **investigate faulty voltage setpoints** with the help of the **network explorer widget**.
- It compares **target voltage** and **calculated voltage** on each generator.
- We will visualize differences by providing the widget a **custom styling** to apply on bus nodes.

In [ ]:
%pip install pypowsybl
%pip install pypowsybl_jupyter
%pip install matplotlib

#### 1. Loading networks
- We load a network snapshot based on the IEEE 300 network.
- We then run an AC loadflow on that snapshot.

In [ ]:
import pypowsybl.network as pn
network = pn.create_ieee300()

In [ ]:
import pypowsybl.loadflow as lf
lf.run_ac(network)

#### 2. Compute voltage differences
- We're constructing the dataframe containing the delta between target_v and calculated v for each generator.
- We're then aggregating that dataframe to have the maximum difference for each bus.

- getting the generators dataframe

In [ ]:
gens = network.get_generators(attributes=['target_v', 'bus_id'])
gens

- getting the buses dataframe

In [ ]:
bus_v = network.get_buses(attributes=['v_mag'])
bus_v

- constructing the `delta_v` column

In [ ]:
gens['delta_v'] = gens['bus_id'].map(bus_v['v_mag']) - gens['target_v']
gens

- looking at extreme values

In [ ]:
gens['delta_v'].min(), gens['delta_v'].max()

In [ ]:
gens.sort_values(by='delta_v', key=abs, ascending=False).head(10)

- Taking the maximum value for each bus to aggregate the values (maximum is taken in absolute value)

In [ ]:
bus_max_delta_v = gens.groupby('bus_id')['delta_v'].apply(lambda s: s.iloc[s.abs().values.argmax()]).to_frame()
bus_max_delta_v['v_mag'] = bus_v['v_mag'][bus_max_delta_v.index]
bus_max_delta_v

#### 3. Construct a style dataframe
  - Green/Red gradient indicate small/big changes
  - Gray indicate NaN values (buses without generators)

In [ ]:
import pandas as pd
import numpy as np

red_value_legend = 0.005

def create_style_gradient(n_colors=100):
    r = np.linspace(91, 255, n_colors)
    g = np.linspace(217, 0, n_colors)
    b = np.linspace(179, 0, n_colors)

    colors = ['#{:02x}{:02x}{:02x}'.format(int(r[i]), int(g[i]), int(b[i])) 
              for i in range(n_colors)]

    numeric_widths = np.linspace(1, 15, n_colors)
    width_strings = [f"{int(width)}px" for width in numeric_widths]

    return pd.DataFrame({
        'width': width_strings,
        'hex_color': colors
    })

style_df = create_style_gradient(100)

def map_index(value):
    if pd.isna(value):
        return -1
    return min(int(abs(value) / red_value_legend * 99), 99)


bus_max_delta_v['dv_idx'] = bus_max_delta_v['delta_v'].apply(map_index)
bus_max_delta_v

#### 4. Construct the custom style profile

- first taking the default style profile

In [ ]:
default_pf = network.get_default_nad_profile()

- then overriding the bus nodes styles based on the `delta_v` column and on the style dataframe

In [ ]:
color = bus_max_delta_v['dv_idx'].apply(lambda idx: 'gray' if idx == -1 else style_df['hex_color'].iloc[idx])
width = bus_max_delta_v['dv_idx'].apply(lambda idx: '1px' if idx == -1 else style_df['width'].iloc[idx])
bus_nodes_styles_df = pd.DataFrame({
    'fill': color
})
bus_nodes_styles_df.index.name = 'id'
bus_nodes_styles_df

- overriding the bus description to display it only for buses for generators, with
   - the calculated v
   - the maximum delta_v

In [ ]:
bus_descriptions_df = pd.DataFrame({
    'description': bus_max_delta_v['v_mag'].apply(lambda x: f"{x:.3f}kV")
                   + ' / '
                   + bus_max_delta_v['delta_v'].apply(lambda x: f"max delta_v={x*1e3:.1f}V")
})
bus_descriptions_df

- remove unneeded branch labels and voltage level descriptions

In [ ]:
vl_descriptions_df = default_pf.vl_descriptions[default_pf.vl_descriptions['type'] != 'FOOTER']
labels_df = pd.DataFrame()

- creating the custom NAD style profile

In [ ]:
diagram_profile=pn.NadProfile(branch_labels=labels_df, vl_descriptions=vl_descriptions_df, bus_descriptions=bus_descriptions_df,
                                      bus_node_styles=bus_nodes_styles_df, edge_styles=default_pf.edge_styles)

#### 5. Display the results!

##### _Construct a colorbar to display the color legend_

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.patches import Patch

def display_legend(max_value):
    cmap = ListedColormap(style_df["hex_color"].tolist())
    norm = Normalize(vmin=0.0, vmax=max_value)
    
    fig, ax = plt.subplots(figsize=(6, 1.2))
    fig.subplots_adjust(bottom=0.5)
    
    sm = ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array([])

    cbar = plt.colorbar(sm, ax=ax, orientation="horizontal", fraction=0.3, pad=0.25)
    cbar.set_label("|delta_v|")

    tick_vals = np.linspace(0, max_value, 5)
    cbar.set_ticks(tick_vals)
    cbar.set_ticklabels([f"{v*1e3:.2f} V" for v in tick_vals])  # convert kV -> V for readability

    ax.axis("off")

    ax.legend(handles=[Patch(facecolor="gray", edgecolor="none", label="NaN / unavailable")],
              loc="upper center", bbox_to_anchor=(0.5, 1.6), ncol=1, frameon=False)

    plt.show()

##### _Display the widget_

In [ ]:
from pypowsybl_jupyter import network_explorer

display_legend(red_value_legend)
network_explorer(network, depth=10, nad_profile=diagram_profile)